In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img, ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
from tqdm.keras import TqdmCallback

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2



In [6]:
data_dir = "../data/cropped"

In [7]:

def create_dataframe(dir):
    image_paths = []
    card_type = []

    for cardtype in os.listdir(dir):
        cardtype_path = os.path.join(dir, cardtype)

        if os.path.isdir(cardtype_path):
            for filename in os.listdir(cardtype_path):
                if filename.endswith('jpg'):
                    image_paths.append(os.path.join(dir, cardtype, filename))
                    # splitting by shading
                    card_type.append(cardtype[1:2])
    return image_paths, card_type

In [8]:
data = pd.DataFrame()
data['images'], data['card type'] = create_dataframe(data_dir)

In [9]:
def reshape_images(images):
    reshaped = []
    for image in tqdm(images):
        img = load_img(image)
        img = np.array(img)
        reshaped.append(img)
    reshaped = np.array(reshaped)
    reshaped = reshaped.reshape(len(reshaped), 200, 150, 3)

    return reshaped

In [10]:
data_reshaped = reshape_images(data['images'])

x = data_reshaped / 255.0

  0%|          | 0/2307 [00:00<?, ?it/s]

In [11]:
le = LabelEncoder()

le.fit(data['card type'])


y = le.transform(data['card type'])
y = to_categorical(y, num_classes=3)

In [12]:
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (2307, 200, 150, 3)
Shape of y: (2307, 3)


In [18]:
model = Sequential()

model.add(Input(shape=(200, 150, 3)))
model.add(Conv2D(32, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size = (5, 5), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.4))

In [19]:
model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.4))

model.add(Dense(3, activation= "softmax"))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [20]:
checkpoint = ModelCheckpoint("../models/number_downsized.keras", monitor = "val_accuracy", verbose = 1, save_best_only = True, mode = 'max')

early_stopping = EarlyStopping(monitor = "val_loss", min_delta = 0.001, patience = 5, verbose = 1, restore_best_weights = True)

reduced_learning_rate = ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3, verbose = 1, min_delta = 0.0001)


callbacks_list = [checkpoint, early_stopping, reduced_learning_rate]

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [22]:
history = model.fit(x = x_train, y = y_train, batch_size = 32, epochs = 50, validation_data = (x_test, y_test), callbacks = callbacks_list)

Epoch 1/50
58/58 [==============================] - ETA: 0s - loss: 1.2477 - accuracy: 0.3350
Epoch 1: val_accuracy improved from -inf to 0.40260, saving model to ../models/number_downsized.keras
58/58 [==============================] - 22s 376ms/step - loss: 1.2477 - accuracy: 0.3350 - val_loss: 1.0984 - val_accuracy: 0.4026 - lr: 0.0010
Epoch 2/50
58/58 [==============================] - ETA: 0s - loss: 1.0992 - accuracy: 0.3144
Epoch 2: val_accuracy did not improve from 0.40260
58/58 [==============================] - 22s 370ms/step - loss: 1.0992 - accuracy: 0.3144 - val_loss: 1.0992 - val_accuracy: 0.3139 - lr: 0.0010
Epoch 3/50
58/58 [==============================] - ETA: 0s - loss: 1.0985 - accuracy: 0.3393
Epoch 3: val_accuracy did not improve from 0.40260
58/58 [==============================] - 21s 370ms/step - loss: 1.0985 - accuracy: 0.3393 - val_loss: 1.0993 - val_accuracy: 0.3139 - lr: 0.0010
Epoch 4/50
58/58 [==============================] - ETA: 0s - loss: 1.0992 - ac

In [14]:
from keras.models import load_model

In [15]:
model = load_model('../models/number_downsized.keras')

In [16]:
def prep(image):
    img = load_img(image)
    feature = np.array(img)
    feature = feature.reshape(1, 200, 150, 3)
    return feature / 255.0

In [17]:
def predict_number(imagepath):
    image = prep(imagepath)
    pred = model.predict(image).argmax()

    if pred == 0:
        return "One", 0
    if pred == 1: 
        return "Two", 1
    else:
        return "Three", 2


In [22]:
imagepath = '../cropped/1111/10.jpg'

pred = predict_number(imagepath)
print("prediction: ", pred[0])

loaded_image = cv2.imread(imagepath)
cv2.imshow('image ', loaded_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

1/1 [==============================] - 0s 17ms/step
prediction:  One
